In [59]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import scipy.io as scipio
import matplotlib.pyplot as plt
%matplotlib inline
import plotly as ly
import datetime
import mapbox
import folium

In [60]:
## Load date and holiday data
date_info = pd.DataFrame(pd.read_csv('data/date_info.csv'))
## Load air data
air_reserve = pd.DataFrame(pd.read_csv('data/air_reserve.csv'))
air_store = pd.DataFrame(pd.read_csv('data/air_store_info.csv'))
air_visit = pd.DataFrame(pd.read_csv('data/air_visit_data.csv'))
## AirREGI Restaurants with Nearest Active Station - add second and third
air_nearest_weather = pd.DataFrame(pd.read_csv('data/rrv-weather-data/air_store_info_with_nearest_active_station.csv'))
## add second and third closest distances
station_distances = pd.DataFrame(pd.read_csv('data/rrv-weather-data/air_station_distances.csv'))

In [61]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


## Restaurant Data 

3 closest weather stations, genre, latlong, and area name

In [62]:
air_nearest_weather.head()
air_byrestindex = air_nearest_weather.set_index(air_nearest_weather["air_store_id"],drop=True)
air_byrestindex.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
air_store_id,,,,,,,,,,,,
air_0f0cdeee6c9bf3d7,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_7cc17a324ae5c7dc,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_fee8dcf4d619598e,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_a17f0778617c76e2,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
air_83db5aff8f50478e,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835


In [5]:
len(air_store)

829

In [6]:
len(air_nearest_weather)

829

In [7]:
station_latlong = station_distances[["station_id", "station_latitude", "station_longitude"]]
# station_latlong = station_latlong.set_index(station_latlong["station_id"])
# station_latlong.head()

In [8]:
from vincenty import vincenty

station_lats = np.array(station_latlong["station_latitude"])
station_longs = np.array(station_latlong["station_longitude"])
station_latlong["latlong_m"] = list(np.vstack((station_lats, station_longs)).T)

/Users/derin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [9]:
rest_lats = np.array(air_nearest_weather["latitude"])
rest_longs = np.array(air_nearest_weather["longitude"])
air_nearest_weather["latlong_m"] = list(np.vstack((rest_lats, rest_longs)).T)

In [10]:
station_latlong["air_fee8dcf4d619598e"] = station_latlong["latlong_m"].map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"][air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"].index[0]]))



/Users/derin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [11]:
# station_latlong["air_fee8dcf4d619598e"] = station_latlong["latlong_m"].map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_fee8dcf4d619598e"]["latlong_m"][0]))

def get_distances(station_data, rest_data):
    for rest in list(rest_data["air_store_id"])[-300:-1]:
        station_data[rest] = station_data["latlong_m"].\
        map(lambda x: vincenty(x, rest_data[rest_data["air_store_id"]==rest]["latlong_m"][rest_data[rest_data["air_store_id"]==rest]["latlong_m"].index[0]]))
    return station_data

# get_distances(station_latlong, air_nearest_weather)


In [12]:
station_latlong["air_c8fe396d6c46275d"] = station_latlong["latlong_m"].\
        map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d"]["latlong_m"][air_nearest_weather[air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d"]["latlong_m"].index[0]]))
# station_latlong["air_0f0cdeee6c9bf3d7"] = station_latlong["latlong_m"].\
#         map(lambda x: vincenty(x, air_nearest_weather[air_nearest_weather["air_store_id"]=="air_0f0cdeee6c9bf3d7"]["latlong_m"][air_nearest_weather[air_nearest_weather["air_store_id"]=="air_0f0cdeee6c9bf3d7"]["latlong_m"].index[0]]))
    
    

/Users/derin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [13]:
station_latlong.to_csv("station_rest_distances.csv")

In [15]:
## filter this
station_rest_latlong = pd.DataFrame(pd.read_csv("station_rest_distances.csv"))
station_rest_latlong.head()

,Unnamed: 0,station_id,station_latitude,station_longitude,latlong_m,air_fee8dcf4d619598e,air_c8fe396d6c46275d
0,0,aichi__ai-xi-kana__isaai,35.216667,136.698333,[ 35.21666667 136.69833333],148.760474,957.952227
1,1,aichi__aki-kana__azo,35.078333,137.413333,[ 35.07833333 137.41333333],206.934423,948.288407
2,2,aichi__centrair-kana__centrair,34.858333,136.805000,[ 34.85833333 136.805 ],148.219287,990.897462
3,3,aichi__chapel-mountain-kana__chaw-yama,35.220000,137.660000,[ 35.22 137.66],232.290097,926.006772
4,4,aichi__gamagori-kana__gamagori,34.845000,137.216667,[ 34.845 137.21666667],185.551102,978.751268


In [16]:
air_nearest_weather = air_nearest_weather.drop(air_nearest_weather["air_store_id"]=="air_c8fe396d6c46275d",0)

In [21]:
# air_nearest_weather = air_nearest_weather.drop(air_nearest_weather["air_store_id"]=="air_a17f0778617c76e2",0)

In [20]:
# Sort the data by the station distances (entries) for each restaurant ID (columns), find the corresponding station
# air_nearest_weather["1_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[0])
# air_nearest_weather["2_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[1])
# air_nearest_weather["3_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[2])
# air_nearest_weather["4_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[3])
# air_nearest_weather["5_close_st"] = air_nearest_weather["air_store_id"].map(lambda x: list(station_rest_latlong[["station_id",str(x)]].sort_values(by=x)["station_id"])[4])

In [116]:
air_nearest_weather = air_nearest_weather.drop(["latitude_str", "longitude_str", "station_id", "station_latitude",\
                                               "station_longitude", "station_vincenty", "station_great_circle"], 1)

In [118]:
air_nearest_weather.to_csv("air_nearest_weather_processed.csv")

## You can start reading the restaurant data from here

In [63]:
import pandas as pd
import numpy as np
air_rest_with_stations = pd.DataFrame(pd.read_csv("air_nearest_weather_processed.csv"))

In [64]:
air_rest_with_stations = air_rest_with_stations.set_index("air_store_id")

In [65]:
air_rest_with_stations.head()

,Unnamed: 0,air_genre_name,air_area_name,latitude,longitude,latlong_m,1_close_st,2_close_st,3_close_st,4_close_st,5_close_st
air_store_id,,,,,,,,,,,
air_fee8dcf4d619598e,2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,[ 34.6951242 135.1978525],hyogo__kobe-kana__koube,hyogo__kobe-airport-kana__kobuteuko,hyogo__rokko-mountain-kana__rokkou-san____2006...,hyogo__ashiya-kana__ashiya____2004-03-01,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...
air_a17f0778617c76e2,3,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,[ 34.6951242 135.1978525],hyogo__kobe-kana__koube,hyogo__kobe-airport-kana__kobuteuko,hyogo__rokko-mountain-kana__rokkou-san____2006...,hyogo__ashiya-kana__ashiya____2004-03-01,hyogo__kobe-shioya-kana__kouteoshaoya____2004-...
air_83db5aff8f50478e,4,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
air_99c3eae84130c1cb,5,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
air_f183a514cb8ff4fa,6,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,[ 35.6580681 139.7515992],tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi


## Daily Visitor Data

indexed by rest_id and days. Cols include day_of_week, holiday_indicator, weather_measurements (multiple TBD), num_visitors

In [66]:
air_visit.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [67]:
date_info = date_info.set_index("calendar_date")

In [68]:
air_visit["day_of_week"] = air_visit["visit_date"].map(lambda x: date_info["day_of_week"][x])
air_visit["holiday_flg"] = air_visit["visit_date"].map(lambda x: date_info["holiday_flg"][x])

In [69]:
air_visit = air_visit[air_visit["air_store_id"]!="air_7cc17a324ae5c7dc"]

In [70]:
air_visit = air_visit[air_visit["air_store_id"]!="air_0f0cdeee6c9bf3d7"]

In [71]:
air_visit["1_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["1_close_st"][x])
air_visit["2_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["2_close_st"][x])
air_visit["3_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["3_close_st"][x])
air_visit["4_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["4_close_st"][x])
air_visit["5_closest_stat"] = air_visit["air_store_id"].map(lambda x: air_rest_with_stations["5_close_st"][x])

In [72]:
## Air_visit_try starts from here

In [73]:
air_visit_try = pd.DataFrame(pd.read_csv("air_visit_wdates.csv"))
air_visit_try.head()

,Unnamed: 0,air_store_id,visit_date,visitors,day_of_week,holiday_flg,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat
0,2,air_ba937bf13d40fb24,2016-01-15,29,Friday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
1,3,air_ba937bf13d40fb24,2016-01-16,22,Saturday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
2,4,air_ba937bf13d40fb24,2016-01-18,6,Monday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
3,5,air_ba937bf13d40fb24,2016-01-19,9,Tuesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi
4,6,air_ba937bf13d40fb24,2016-01-20,31,Wednesday,0,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__haneda-kana__haneda,tokyo__setagaya-kana__setagaya,kanagawa__hiyoshi-kana__hiyoshi


## Merging for Weather Measurements

Will take into account the data of all restaurants as separate DataFrames, then merge each of them with the measurements of corresponding stations, then concat all of them along with the y-axis

In [33]:
## trying
air_ba937bf13d40fb24_df = (pd.DataFrame(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]).\
join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(air_visit_try[air_visit_try["air_store_id"]=="air_ba937bf13d40fb24"]["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="0", rsuffix="1")).drop(["Unnamed: 00", "Unnamed: 01"],1)

In [8]:
def merge_rest_with_weather_data(data, rest_id):
    merged_data = (((((pd.DataFrame(air_visit_try[air_visit_try["air_store_id"]==rest_id]).\
join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="0", rsuffix="1")).drop(["Unnamed: 00", "Unnamed: 01"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1]  + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="1", rsuffix="2").drop(["Unnamed: 0"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="2", rsuffix="3").drop(["Unnamed: 0"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="3", rsuffix="4").drop(["Unnamed: 0"],1)).join(pd.DataFrame(pd.read_csv("data/rrv-weather-data/measurements/"+list(pd.DataFrame(data[data["air_store_id"]==rest_id])["1_closest_stat"])[-1] + ".csv")).set_index("calendar_date"),\
    on="visit_date", how="left",lsuffix="4", rsuffix="5").drop(["Unnamed: 0"],1)
    return merged_data

In [9]:
## merging the separate dataframes that belong to each restaurant with corresponding 5 weather
## center measurement data, and writing all in a dictionary where "rest_id" = dataframe object
df_weather = {}
for rest in list(set(air_visit_try["air_store_id"])):
    df_weather[rest] = merge_rest_with_weather_data(air_visit_try, rest)

In [74]:
# df_weather["air_c8a657c8c5c93d69"].columns.values

In [53]:
def select_weather_measurement(data):
    """Given a rest_id dataframe merged with weather measurements, 
    selects the weather measurement columns starting from the closest station.
    Final measurement features should include at least one column from each each 
    variable below: 
    - low_temperature,
    - high_temperature,
    - avg_temperature,
    - hours_sunlight."""
    ## The measurement types that are wanted in the data (from at least one station)
    required_measurements = ["hours_sunlight", "low_temperature", "high_temperature",\
                            "avg_temperature", "precipitation","avg_wind_speed",
                            "avg_local_pressure", "avg_sea_pressure","avg_vapor_pressure",
                            "avg_humidity", "cloud_cover"]
    ## The core columns that should not be dropped before returning the data
    ## These are not weather measurements.
    core_columns = ["air_store_id", "visit_date", "visitors", "day_of_week",\
                   "holiday_flg", "1_closest_stat", "2_closest_stat", "3_closest_stat",\
                   "4_closest_stat", "5_closest_stat"]
    
    ## Initialize the list of column names that will be kept in the data
    for col in required_measurements:
        ## Check if the closest station has that measurement
        ### 
        ### Infer these from the data
        ###
        if str(col) + "1" in data.columns.values and \
        (len(data[data[str(col) + "1"].isnull() == False])<3000):
            core_columns.append(str(col + "1"))
        else:
#             print ("Too many NaNs in col1")
            if str(col) + "2" in data.columns.values and \
        (len(data[data[str(col) + "2"].isnull() == False])<3000):
                core_columns.append(str(col + "2"))
            else:
#                 print ("Too many NaNs in col2")
                if str(col) + "3" in data.columns.values and \
        (len(data[data[str(col) + "3"].isnull() == False])<3000):
                    core_columns.append(str(col + "3"))
                else:
#                     print ("Too many NaNs in col3")
                    if str(col) + "4" in data.columns.values and \
        (len(data[data[str(col) + "4"].isnull() == False])<3000):
                        core_columns.append(str(col + "4"))
                    else:
#                         print ("Too many NaNs in col4")
                        if str(col) + "5" in data.columns.values and \
        (len(data[data[str(col) + "5"].isnull() == False])<3000):
                            core_columns.append(str(col + "5"))
                        elif col in data.columns.values:
                            core_columns.append(col)

    return data[core_columns]

In [54]:
# selected = {}
# selected_all = {}
# for rest in list(df_weather.keys()):
#     selected[rest] = select_weather_measurement(df_weather[rest])
#     if ("hours_sunlight" or "hours_sunlight1" or "hours_sunlight2"or "hours_sunlight3" or "hours_sunlight4" in selected[rest].columns.values) and ("low_temperature" or "low_temperature1" or "low_temperature2" or "low_temperature3"or "low_temperature4" in selected[rest].columns.values)\
#     and ("high_temperature" or "high_temperature1" or "high_temperature2" or "high_temperature3" or "high_temperature4" in selected[rest].columns.values) and ("avg_temperature" or "avg_temperature1" or "avg_temperature2" or "avg_temperature3" or "avg_temperature4" in selected[rest].columns.values)\
#     and ("precipitation" or "precipitation1" or "precipitation2" or "precipitation3" or "precipitation4" in selected[rest].columns.values):
#         selected_all[rest] = selected[rest]

In [55]:
merged_data_final = pd.concat([select_weather_measurement(df_weather[rest]) for rest in df_weather.keys()])

In [56]:
merged_data_final

,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,air_store_id,avg_humidity1,avg_local_pressure1,avg_sea_pressure1,avg_temperature1,...,avg_wind_speed1,cloud_cover1,day_of_week,high_temperature1,holiday_flg,hours_sunlight1,low_temperature1,precipitation1,visit_date,visitors
153134,fukuoka__hakata-kana__hakata,fukuoka__fukuoka-kana__fukuoka,fukuoka__dazaifu-kana__dasaif,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__sasaguri-kana__sasaburi____2009-11-20,air_2d3afcb91762fe01,NaN,NaN,NaN,10.5,...,2.4,NaN,Monday,15.9,0,NaN,6.2,0.0,2016-01-04,67
153135,fukuoka__hakata-kana__hakata,fukuoka__fukuoka-kana__fukuoka,fukuoka__dazaifu-kana__dasaif,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__sasaguri-kana__sasaburi____2009-11-20,air_2d3afcb91762fe01,NaN,NaN,NaN,9.4,...,1.5,NaN,Tuesday,12.9,0,NaN,7.5,1.0,2016-01-05,37
153136,fukuoka__hakata-kana__hakata,fukuoka__fukuoka-kana__fukuoka,fukuoka__dazaifu-kana__dasaif,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__sasaguri-kana__sasaburi____2009-11-20,air_2d3afcb91762fe01,NaN,NaN,NaN,8.7,...,1.5,NaN,Wednesday,12.4,0,NaN,5.3,0.0,2016-01-06,52
153137,fukuoka__hakata-kana__hakata,fukuoka__fukuoka-kana__fukuoka,fukuoka__dazaifu-kana__dasaif,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__sasaguri-kana__sasaburi____2009-11-20,air_2d3afcb91762fe01,NaN,NaN,NaN,8.8,...,3.9,NaN,Thursday,10.1,0,NaN,7.0,0.0,2016-01-07,15
153138,fukuoka__hakata-kana__hakata,fukuoka__fukuoka-kana__fukuoka,fukuoka__dazaifu-kana__dasaif,fukuoka__sayara-wakayama-kana__sawarawakiyama,fukuoka__sasaguri-kana__sasaburi____2009-11-20,air_2d3afcb91762fe01,NaN,NaN,NaN,7.3,...,5.0,NaN,Friday,8.6,0,NaN,6.3,0.0,2016-01-08,21


In [57]:
merged_data_final = pd.concat([select_weather_measurement(df_weather[rest]) for rest in df_weather.keys()])
# select_weather_measurement(df_weather["air_ba937bf13d40fb24"])
# merged_data_final.to_csv("merged_rest_data_weather_measurements.csv")

In [47]:
# merged_data_final[["avg_humidity1", "avg_local_pressure1", "avg_sea_pressure1",
#                   "avg_temperature1", "avg_vapor_pressure1", "avg_wind_speed1",
#                   "cloud_cover1"]]

In [50]:
# len(merged_data_final[merged_data_final["avg_vapor_pressure1"].isnull() == False])


In [ ]:
import pandas as pd
# merged_data_final = pd.read_csv("merged_rest_data_weather_measurements.csv")
merged_data_final

In [2]:
import pandas as pd
pd.DataFrame(pd.read_csv("merged_rest_data_weather_measurements.csv")) ## send to rong

,Unnamed: 0,1_closest_stat,2_closest_stat,3_closest_stat,4_closest_stat,5_closest_stat,air_store_id,avg_temperature1,day_of_week,high_temperature1,holiday_flg,hours_sunlight1,low_temperature1,precipitation1,visit_date,visitors
0,228949,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,9.3,Sunday,16.2,1,8.3,3.0,NaN,2016-01-03,10
1,228950,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,9.2,Monday,15.2,0,8.9,4.4,NaN,2016-01-04,9
2,228951,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,10.9,Tuesday,15.4,0,8.4,5.1,NaN,2016-01-05,17
3,228952,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,8.9,Wednesday,11.3,0,0.0,7.1,NaN,2016-01-06,10
4,228953,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,6.8,Friday,10.9,0,8.7,3.1,NaN,2016-01-08,5
5,228954,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,7.3,Saturday,11.6,0,8.9,2.8,NaN,2016-01-09,8
6,228955,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,7.9,Sunday,12.9,0,9.1,2.4,NaN,2016-01-10,16
7,228956,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,7.1,Monday,10.5,1,3.3,3.8,NaN,2016-01-11,16
8,228957,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,3.4,Tuesday,6.1,0,0.0,0.9,NaN,2016-01-12,8
9,228958,tokyo__tokyo-kana__tonokyo,tokyo__edogawa-seaside-kana__edgawawinkai,tokyo__nerima-kana__nerima,tokyo__setagaya-kana__setagaya,tokyo__haneda-kana__haneda,air_0241aa3964b7f861,4.3,Wednesday,9.7,0,8.7,-0.9,NaN,2016-01-13,9
